# Export workflow definitions

We need to compose workflows and place the function prototypes into their proper place.

### IMports


In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json


In [ ]:
import infer_subc_2d.constants

-----------------
## WORKFLOWS



-----------------
## NUCLEI workflow

Write the `infer_nuclei_fromlabel` spec to the widget json 

In [ ]:
from infer_subc_2d.constants import NUC_CH


def make_infer_nuclei_dict():
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_1.1.nuclei", infer_nuclei_dict)

_____________

## CELLMASK

In [ ]:
def make_infer_cellmask_fromaggr_step_by_step_dict():
    """
    crete .json version of infer_cellmask_fromaggr
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
                                
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    step_n += 1                     
    nuc_step = step_n

    step_name.append(f"{step_n}")
    function_name.append("raw_cellmask_fromaggr")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_input_step = step_n

    step_n += 1                     
    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(cellmask_input_step)
    annotation.append(f"cellmask segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_cellmask_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"cellmask segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(step_n-1)
    annotation.append(f"cellmask segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"cellmask segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_cellmask_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(None)
    annotation.append(f"cellmask segmentation: {step_n}")
    parent.append([struct_img_step, step_n-1,nuc_step])

    soma_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, soma_step])
    annotation.append(f"infer cytoplasm: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_cellmask_fromaggr_stepbystep_dict = make_infer_cellmask_fromaggr_step_by_step_dict()
infer_cellmask_fromaggr_stepbystep_dict

write_workflow_json("conf_1.2.cellmask", infer_cellmask_fromaggr_stepbystep_dict)

In [ ]:
from infer_subc_2d.constants import NUC_CH

def make_infer_cellmask_fromaggr_step_by_step_dict2():
    """
    crete .json version of infer_cellmask_fromaggr
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0
    step_n = 1

    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n

    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("make_aggregate")
    category.append("extraction")
    parameter_values.append(dict(w0=0,
                                    w1=6,
                                    w2=0,
                                    w3=2,
                                    w4=0,
                                    w5=1,
                                    w6=0,
                                    w7=0,
                                    w8=0,
                                    w9=0,
                                    scale_min_max = True) 
                )
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_input_step = step_n

    step_n += 1                   

    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(cellmask_input_step)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_cellmask_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_cellmask_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(None)
    annotation.append(f"basic cellmask segmentation: {step_n}")
    parent.append([struct_img_step, step_n-1,nuc_step])
    soma_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, soma_step])
    annotation.append(f"infer cytoplasm: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_cellmask_fromaggr_stepbystep_dict2 = make_infer_cellmask_fromaggr_step_by_step_dict2()

write_workflow_json("conf_1.2.cellmask2", infer_cellmask_fromaggr_stepbystep_dict2)

--------------------------
## CYTOPLASM .json


In [ ]:
def make_infer_cytoplasm_dict():
    """
    Procedure to infer cyto from linearly unmixed input. (logical cellmask AND NOT nucleus)
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei_fromlabel")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n
    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("infer_cellmask_fromaggr_MCZ")
    category.append("extraction")
    parameter_values.append(dict(
                                                    median_sz = 15,
                                            gauss_sig = 1.34,
                                            mo_method = "ave_tri_med",
                                            mo_adjust = 0.5,
                                            mo_cutoff_size = 150,
                                            max_hole_w = 50,
                                            small_obj_w = 45
                                            ))
    parent.append([raw_input_step, nuc_step])
    annotation.append(f" this creates an aggregate signal for the cellmask" )
    cellmask_step = step_n

    step_n += 1                     
    step_name.append(f"{step_n}")
    function_name.append("infer_cytoplasm")
    category.append("core")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, cellmask_step])
    annotation.append(f"basic cellmask segmentation: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_cytoplasm_dict = make_infer_cytoplasm_dict()

write_workflow_json("conf_1.3.cytoplasm", infer_cytoplasm_dict)

-------------------------------
## LYSOSOME

In [ ]:
from infer_subc_2d.constants import LYSO_CH
def make_infer_lyso_step_by_step_from_raw_dict():
    """
    Procedure to infer lysosome from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LYSO_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic lyso segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic lyso segmentation: {step_n}")
    img_step = step_n
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 5,
                                dot_cut_1 = 0.09,
                                dot_scale_2 = 2.5,
                                dot_cut_2 = 0.07,
                                dot_scale_3 = 1,
                                dot_cut_3 = 0.01))
    parent.append(step_n-1)
    annotation.append(f"basic lyso - spot filter: {step_n}")
    spot_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("filament_filter")
    category.append("core")
    parameter_values.append(dict(  filament_scale = 1.,
                                                        filament_cut = 0.15))
    parent.append(img_step)
    annotation.append(f"basic lyso - filament filter: {step_n}")
    fil_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([spot_step,fil_step])
    annotation.append(f"basic lyso - combine spot+filament: {step_n}")
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic lyso - fill/filter: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        



    return out_dict

In [ ]:

infer_lyso_stepbystep_from_raw_dict = make_infer_lyso_step_by_step_from_raw_dict()

write_workflow_json("conf_1.4.lyso", infer_lyso_stepbystep_from_raw_dict)

-------------------------------
## MITO

In [ ]:
from infer_subc_2d.constants import MITO_CH 

def make_infer_mito_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = MITO_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic mito segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic mito segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("vesselness_slice_by_slice")
    category.append("core")
    parameter_values.append( dict( sigma=1.5, cutoff=0.05))
    parent.append(step_n-1)
    annotation.append(f"basic mito - vesselness filter: {step_n}")
    spot_step = step_n
    step_n += 1



    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic mito - fill/filter: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict



In [ ]:

infer_mito_stepbystep_from_raw_dict = make_infer_mito_step_by_step_from_raw_dict()

write_workflow_json("conf_1.5.mito", infer_mito_stepbystep_from_raw_dict)

-------------------------------
## GOLGI

In [ ]:
from infer_subc_2d.constants import GOLGI_CH
def make_infer_golgi_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = GOLGI_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic golgi segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic golgi segmentation: {step_n}")
    img_step = step_n
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict(th_method= "triangle", 
                                                            cutoff_size=1200,
                                                            th_adjust = 0.5) )
    parent.append(img_step)
    annotation.append(f"basic golgi - mo: {step_n}")
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("topology_preserving_thinning")
    category.append("core")
    parameter_values.append(dict(    
                                                min_thickness = 1.6,
                                                thin = 1) )
    parent.append(step_n-1)
    annotation.append(f"basic golgi - thinning filter: {step_n}")
    fil_step = step_n
    step_n += 1
    
    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 1.6,
                                dot_cut_1 = 0.02,
                                dot_scale_2 = 0,
                                dot_cut_2 = 0.1,
                                dot_scale_3 = 0,
                                dot_cut_3 = 0.1))
    parent.append(img_step)
    annotation.append(f"basic golgi - spot filter: {step_n}")
    spot_step = step_n
    step_n += 1


    step_name.append(f"{step_n}")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([spot_step,fil_step])
    annotation.append(f"basic golgi - combine spot+thinned: {step_n}")

    step_n += 1



    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic mito - fill/filter: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict



In [ ]:

infer_golgi_stepbystep_from_raw_dict = make_infer_golgi_step_by_step_from_raw_dict()

write_workflow_json("conf_1.6.golgi", infer_golgi_stepbystep_from_raw_dict)

-------------------------------
## PEROXISOME

In [ ]:
from infer_subc_2d.constants import PEROX_CH


def make_infer_perox_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = PEROX_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic perox segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 0, gauss_sig=3.0 ))
    parent.append(step_n-1)
    annotation.append(f"basic perox segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################

    step_name.append(f"{step_n}")
    function_name.append("spot_filter_3")
    category.append("core")
    parameter_values.append(dict( dot_scale_1 = 1.0,
                                dot_cut_1 = 0.01,
                                dot_scale_2 = 0,
                                dot_cut_2 = 0.1,
                                dot_scale_3 = 0,
                                dot_cut_3 = 0.1))
    parent.append(step_n-1)
    annotation.append(f"basic perox - spot filter (1 scale): {step_n}")
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 0, 
                                  min_size = 2,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic perox - fill/filter: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict




In [ ]:

infer_perox_stepbystep_from_raw_dict = make_infer_perox_step_by_step_from_raw_dict()

write_workflow_json("conf_1.7.perox", infer_perox_stepbystep_from_raw_dict)

-------------------------------
## ER


In [ ]:
from infer_subc_2d.constants import ER_CH
def make_infer_er_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    ###################
    # EXTRACT
    ###################   
    step_n = 1
    raw_input_step = 0

    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = ER_CH) )
    parent.append(raw_input_step)
    annotation.append(f"basic ER segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 0, gauss_sig=3.0 ))
    parent.append(step_n-1)
    annotation.append(f"basic ER segmentation: {step_n}")
    step_n += 1


    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("filament_filter")
    category.append("core")
    parameter_values.append(dict(  filament_scale = 1.,
                                                        filament_cut = 0.015))
    parent.append(step_n-1)
    annotation.append(f"basic ER - filament filter: {step_n}")
    fil_step = step_n
    step_n += 1


    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 0, 
                                  min_size = 2,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic ER - fill/filter: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    
    return out_dict




In [ ]:

infer_er_stepbystep_from_raw_dict = make_infer_er_step_by_step_from_raw_dict()

write_workflow_json("conf_1.8.ER", infer_er_stepbystep_from_raw_dict)

-------------------------------
## LD


In [ ]:
from infer_subc_2d.constants import LD_CH

def make_infer_LD_step_by_step_from_raw_dict():
    """
    Procedure to infer lipid from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    LD_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LD_CH) )
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 0, gauss_sig=2.4 ))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("apply_threshold")
    category.append("core")
    parameter_values.append(dict(method = "otsu",
                                                        thresh_factor = 0.8, 
                                                            thresh_min = .5,
                                                            thresh_max = 1.))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 2.5, 
                                  min_size = 4,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic LD segmentation: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:

infer_LD_stepbystep_from_raw_dict = make_infer_LD_step_by_step_from_raw_dict()

write_workflow_json("conf_1.9.LD", infer_LD_stepbystep_from_raw_dict)